# 📤 Model Upload API with FastAPI
This notebook sets up a local FastAPI server to handle file uploads (e.g., spam detection models) and demonstrates how to upload a file to that endpoint.

In [1]:
# ✅ Install required packages
!pip install fastapi uvicorn requests nest_asyncio

In [2]:
# ⚙️ Setup FastAPI App
# model_validation_dashboard.py

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse
from joblib import load
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

app = FastAPI()

# Dummy test data for validation (replace with actual dataset)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Sample data
X_data = ["Congratulations! You have won a prize!", "Meeting at 10am tomorrow", "Free entry in a weekly contest!", "Lunch with team today", "Win a brand new iPhone now"]
y_data = [1, 0, 1, 0, 1]  # 1=Spam, 0=Ham

vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X_data)
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_data, test_size=0.4, random_state=42)

@app.post("/api/upload", response_class=HTMLResponse)
async def upload_model(model: UploadFile = File(...)):
    model_path = f"uploaded_model.joblib"
    with open(model_path, "wb") as f:
        content = await model.read()
        f.write(content)

    clf = load(model_path)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    cls_report = classification_report(y_test, y_pred, output_dict=True)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Plot Confusion Matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    image_path = "static/conf_matrix.png"
    os.makedirs("static", exist_ok=True)
    plt.savefig(image_path)
    plt.close()

    # Render HTML Dashboard
    report_df = pd.DataFrame(cls_report).T.round(4)
    html_table = report_df.to_html(classes="styled-table")

    html = f"""
    <html>
    <head>
    <style>
        body {{ background-color: #1c1f26; color: #d6ff4f; font-family: Arial; padding: 20px; }}
        h2 {{ color: #d6ff4f; }}
        .styled-table {{ border-collapse: collapse; width: 100%; color: white; }}
        .styled-table th, .styled-table td {{ border: 1px solid #ddd; padding: 8px; }}
        .styled-table th {{ background-color: #333; color: #d6ff4f; }}
        img {{ margin-top: 20px; max-width: 500px; }}
    </style>
    </head>
    <body>
        <h1>Model Validation Feedback</h1>
        <p>Your model accuracy is <strong>{acc:.2f}</strong>. Aim for at least 0.90 to pass benchmark.</p>
        <h2>Detailed Metrics</h2>
        {html_table}
        <h2>Confusion Matrix</h2>
        <img src="/{image_path}" />
    </body>
    </html>
    """
    return HTMLResponse(content=html)

# Run this using: uvicorn model_validation_dashboard:app --reload


In [ ]:
# ▶️ Start the FastAPI server inside the notebook
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [50144]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56690 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56690 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56700 - "GET / HTTP/1.1" 404 Not Found


## 📤 Uploading a File using Python Requests

In [ ]:
import requests
import json

# Define API URL
url = "http://localhost:8000/api/upload"

# Replace with your model file path
model_file_path = "spam_detection_model.joblib"

with open(model_path, "rb") as f:
    res = requests.post(url, files={"model": f})
    print(res.text)